# ESRD association analysis

In [1]:
import pandas as pd
import dxpy
import dxdata
import numpy as np
import re
import shutil
import subprocess
import glob
import os
import ast

In [3]:
#ancestry:
ans = pd.read_csv("/mnt/project/Datasets/Ancestry/panukbb_ancestry.txt", sep="\t", index_col=0)
ans.set_index("individual_id", inplace=True)
ans.index = ans.index.astype(str)
ans = ans[~ans.index.duplicated(keep="first")]

In [4]:
#V1, G1, G2:
from bgen import BgenReader
vname = "22:36265284:G:A"
fn = "/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c22_b0_v1.sample"
sns = pd.read_csv(fn, sep=" ")
sns = sns.iloc[1:,:]
sns = sns.ID_1
fn = "/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c22_b0_v1.bgen"
bfile = BgenReader(fn)
v = bfile.with_rsid(vname)[0]
v1 = pd.Series(v.alt_dosage)
v1.index = sns

vname = "22:36265988:T:G"
fn = "/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c22_b0_v1.sample"
sns = pd.read_csv(fn, sep=" ")
sns = sns.iloc[1:,:]
sns = sns.ID_1
fn = "/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c22_b0_v1.bgen"
bfile = BgenReader(fn)
v = bfile.with_rsid(vname)[0]
g1 = pd.Series(v.alt_dosage)
g1.index = sns
g1.name = "g1m"
g1 = pd.DataFrame(g1)
g1.index = g1.index.astype(str)

vname = "22:36265995:AATAATT:A"
fn = "/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c22_b0_v1.sample"
sns = pd.read_csv(fn, sep=" ")
sns = sns.iloc[1:,:]
sns = sns.ID_1
fn = "/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c22_b0_v1.bgen"
bfile = BgenReader(fn)
v = bfile.with_rsid(vname)[0]
g2 = pd.Series(v.alt_dosage)
g2.index = sns
g2.name = "g2"
g2 = pd.DataFrame(g2)
g2.index = g2.index.astype(str)

/tmp/ipykernel_82/1841932591.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sns = pd.read_csv(fn, sep=" ")
/tmp/ipykernel_82/1841932591.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sns = pd.read_csv(fn, sep=" ")
/tmp/ipykernel_82/1841932591.py:30: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sns = pd.read_csv(fn, sep=" ")


In [5]:
#ESRD:
DATASET_ID = "REDACTED"
dataset = dxdata.load_dataset(id=DATASET_ID)

participant = dataset['participant']
field_eid = participant.find_field(name="eid")
field_esrd = participant.find_field(name="p42026")
esrd = participant.retrieve_fields(engine=dxdata.connect(), fields=[field_eid, field_esrd], coding_values="replace").toPandas()
esrd.set_index("eid", inplace=True)
esrd.columns = ["esrd"]

In [6]:
#Age, sex:
DATASET_ID = "REDACTED"
dataset = dxdata.load_dataset(id=DATASET_ID)
participant = dataset['participant']
field_age = participant.find_field(title="Age at recruitment")
field_sex = participant.find_field(title="Sex")
field_eid = participant.find_field(name="eid")
agesex = participant.retrieve_fields(engine=dxdata.connect(), fields=[field_eid, field_age, field_sex], coding_values="replace").toPandas()
agesex.set_index("eid", inplace=True)
agesex.columns = ["age", "sex"]
agesex["sex"] = agesex.sex.str.replace("Female","0").str.replace("Male","1").astype(int)


In [ ]:
#Merge everything:
v1.name = "v1"
v1.index = v1.index.astype(str)
variants = g1.join(g2, how="left").join(v1, how="left")

In [ ]:
df = variants.join(agesex, how="left").join(ans, how="left").join(esrd, how="left")

In [33]:
df["esrd_case"] = (~df.esrd.isna()).astype(int)
df.esrd_case.value_counts()

0    468169
1      1666
Name: esrd_case, dtype: int64

In [34]:
import statsmodels.api as sm
def logistic_regression_with_covariates(y: pd.Series, x: pd.Series, Z: pd.DataFrame, n_case_thres=10):
    data = pd.concat([x, Z], axis=1)
    data = sm.add_constant(data)
    isna = (data.isna().any(axis=1)|y.isna())
    data = data[~isna]
    y = y[~isna]
    n_case = sum(y)
    n_cont = len(y) - sum(y)
    if n_case<n_case_thres:
        return {'log_odds_ratio': np.nan,'std_err': np.nan, 'pval': np.nan, 'N_case': n_case, "N_cont": n_cont}
    else:
        try:
            model = sm.Logit(y, data).fit(disp=False)
            log_odds_ratio = model.params[x.name]
            std_err = model.bse[x.name]
            p_value = model.pvalues[x.name]
            return {'log_odds_ratio': log_odds_ratio,'std_err': std_err, 'pval': p_value, 'N_case': n_case, "N_cont": n_cont}
        except:
            return {'log_odds_ratio': np.nan,'std_err': np.nan, 'pval': np.nan, 'N_case': n_case, "N_cont": n_cont}

In [36]:
dfe = df[df["pop"]=="EUR"]
y = dfe.esrd_case
x = dfe.v1
z = df[["age", "sex", "PC1", "PC2", "PC3", "PC4", "PC5"]]
out = logistic_regression_with_covariates(y, x, z)

/tmp/ipykernel_82/2272967913.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data[~isna]


In [38]:
pd.Series(out)

log_odds_ratio         0.021153
std_err                0.049916
pval                   0.671733
N_case              1336.000000
N_cont            409035.000000
dtype: float64

In [49]:
#Testing within AFR population:
dfa = df[df["pop"]=="AFR"]
dfa["g1g2_recessive"] = (dfa.g1m+dfa.g2>0).astype(int)
y = dfa.esrd_case
x = dfa.g1g2_recessive
z = dfa[["age", "sex", "PC1", "PC2", "PC3", "PC4", "PC5"]]
is_na = y.isna()|x.isna()|z.isna().sum(axis=1)
x = x[~is_na]
y = y[~is_na]
z = z[~is_na]
out = logistic_regression_with_covariates(y, x, z)
pd.Series(out)
#So it is not even nominally associated..

/tmp/ipykernel_82/235367144.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa["g1g2_recessive"] = (dfa.g1m+dfa.g2>0).astype(int)


log_odds_ratio      -0.216174
std_err              0.254774
pval                 0.396162
N_case              68.000000
N_cont            6341.000000
dtype: float64